In [1]:
#Load Basic Modules
import numpy as np
import pandas as pd

# 0 indicates non-fraudulent claim and 1 indicates fraudulent claim

In [4]:
#Basic ETL
import pandas as pd
import os
import shutil

# Read the CSV file
data = pd.read_csv("C:/Users/aditya/Desktop/2024/Hackathon_AV/train/train.csv")

# Path to the folder containing the images
image_folder = "C:/Users/aditya/Desktop/2024/Hackathon_AV/train/images"

# Create folders for each label
for label in data['label'].unique():
    os.makedirs(os.path.join(image_folder, str(label)), exist_ok=True)

# Move images to their respective folders based on the label
for index, row in data.iterrows():
    image_id = row['image_id']
    filename = row['filename']
    label = row['label']
    src = os.path.join(image_folder, filename)
    dst = os.path.join(image_folder, str(label), filename)
    shutil.move(src, dst)

print("Images moved to respective folders.")


Images moved to respective folders.


# Model -1 Resnet -50

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import os

# Define transform to preprocess the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
train_dataset = ImageFolder(root="images/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# Calculate class weights
class_weights = torch.tensor([len(train_dataset) / (len(train_dataset) - sum(train_dataset.targets)),
                               len(train_dataset) / sum(train_dataset.targets)], dtype=torch.float)

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 output classes (0 and 1)

# Define loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
  
# Save the trained model
torch.save(model.state_dict(), 'resnet50_model.pth')


In [16]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50
import pandas as pd
from PIL import Image
import os
import torch.nn as nn  # Import nn module directly

class CustomTestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_filenames = sorted(os.listdir(root))

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.root, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name  # Return image_name as the image ID

# Define transform for test images
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = CustomTestDataset(root="C:/Users/aditya/Desktop/2024/Hackathon_AV/test/images/.", transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the trained ResNet50 model
model = resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Assuming 2 output classes (0 and 1)
model.load_state_dict(torch.load('resnet50_model.pth'))
model.eval()

# Create a DataFrame to store predictions
predictions = {'image_id': [], 'label': []}

# Predict labels for test images
with torch.no_grad():
    for inputs, image_names in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for pred, image_name in zip(predicted, image_names):
            predictions['image_id'].append(image_name.split('.')[0])  # Extract image ID from the file name
            predictions['label'].append(pred.item())

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save predictions to CSV file
df_predictions.to_csv('submission.csv', index=False)


c:\Users\aditya\anaconda3\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\aditya\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# Score 0.6535516551

# Model -2 VGG-19

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np

# Define transform to preprocess the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
train_dataset = ImageFolder(root="images/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# Calculate class weights
class_counts = np.bincount(train_dataset.targets)
class_weights = torch.tensor(class_counts.sum() / (len(class_counts) * class_counts), dtype=torch.float)

# Load pre-trained VGG-19 model
model = models.vgg19(pretrained=True)
# Replace the classifier
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # 2 output classes (0 and 1)

# Define loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train only the classifier layers (fine-tuning)
for param in model.features.parameters():
    param.requires_grad = False

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'vgg19_model_with_class_weights_fine_tuned.pth')


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vgg19
import pandas as pd
from PIL import Image
import os
import torch.nn as nn  # Import nn module directly

class CustomTestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_filenames = [filename for filename in os.listdir(root) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.root, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name  # Return image_name as the image ID

# Define transform for test images
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = CustomTestDataset(root="test/images/", transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the trained VGG-19 model
model = vgg19(pretrained=False)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # Assuming 2 output classes (0 and 1)
model.load_state_dict(torch.load('vgg19_model_with_class_weights_fine_tuned.pth'))
model.eval()

# Create a DataFrame to store predictions
predictions = {'image_id': [], 'label': []}

# Predict labels for test images
with torch.no_grad():
    for inputs, image_names in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for pred, image_name in zip(predicted, image_names):
            predictions['image_id'].append(image_name.split('.')[0])  # Extract image ID from the file name
            predictions['label'].append(pred.item())

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save predictions to CSV file
df_predictions.to_csv('submission_1.csv', index=False)


# Score - 0.6816959152

# Model -3 VGG19 With augmentation

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ColorJitter

# Define transform to preprocess and augment the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    RandomHorizontalFlip(),
    RandomRotation(10),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
train_dataset = ImageFolder(root="images/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# Calculate class weights
class_counts = np.bincount(train_dataset.targets)
class_weights = torch.tensor(class_counts.sum() / (len(class_counts) * class_counts), dtype=torch.float)

# Load pre-trained VGG-19 model
model = models.vgg19(pretrained=True)
# Replace the classifier
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # 2 output classes (0 and 1)

# Define loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train only the classifier layers (fine-tuning)
for param in model.features.parameters():
    param.requires_grad = False

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'vgg19_model_with_augmentation_class_weights_fine_tuned.pth')


# Score - 0.7148351648 

# Mode based approach

In [1]:
import pandas as pd

# Load the data from the files
file_paths = ["submission.csv", "submission_1 (1).csv", "submission_2.csv"]
data_frames = [pd.read_csv(file) for file in file_paths]

# Concatenate the data frames
merged_df = pd.concat(data_frames)

# Group by image_id and calculate the mode label for each image_id
mode_labels = merged_df.groupby('image_id')['label'].agg(lambda x: x.mode()[0])

# Create a new DataFrame with image_id and mode label
result_df = pd.DataFrame({'image_id': mode_labels.index, 'label': mode_labels.values})

# Save the result to a new CSV file
result_df.to_csv('result.csv', index=False)


# Score - 0.7193250095

# Model -4 Resnet -50 with augmentation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50

# Define transform to preprocess and augment the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
dataset = ImageFolder(root="images/", transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Define DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet50 model
model = resnet50(pretrained=True)

# Freeze layers
for param in model.parameters():
    param.requires_grad = False

# Modify the fully connected layer to match the number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))  # Use len(dataset.classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Early stopping
best_val_loss = float('inf')
patience = 3
counter = 0

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {100.*correct/total:.2f}%")

    # Validate the model
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    val_loss /= len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}, Val Acc: {100.*correct/total:.2f}%")

    # Update learning rate
    scheduler.step()

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping...")
            break

# Save the trained model
torch.save(model.state_dict(), 'resnet50_model.pth')


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50
import pandas as pd
from PIL import Image
import os
import torch.nn as nn  # Import nn module directly

class CustomTestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_filenames = [filename for filename in os.listdir(root) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.root, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name  # Return image_name as the image ID

# Define transform for test images
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = CustomTestDataset(root="test/images/", transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the trained ResNet50 model
model = resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Assuming 2 output classes (0 and 1)
model.load_state_dict(torch.load('resnet50_model.pth'))
model.eval()

# Create a DataFrame to store predictions
predictions = {'image_id': [], 'label': []}

# Predict labels for test images
with torch.no_grad():
    for inputs, image_names in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for pred, image_name in zip(predicted, image_names):
            predictions['image_id'].append(image_name.split('.')[0])  # Extract image ID from the file name
            predictions['label'].append(pred.item())

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save predictions to CSV file
df_predictions.to_csv('submission_resnet50.csv', index=False)


# Score -0.65

# Model  5 Inception V3 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import inception_v3
import numpy as np

# Define transform to preprocess and augment the images
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
dataset = ImageFolder(root="images/", transform=transform)

# Count occurrences of each class
class_counts = np.bincount(dataset.targets)
num_classes = len(class_counts)

# Determine the minority class
minority_class = np.argmin(class_counts)

# Oversample the minority class to balance the dataset
oversampled_indices = []
for idx, (data, target) in enumerate(dataset):
    if target == minority_class:
        oversampled_indices.append(idx)

oversampled_dataset = torch.utils.data.Subset(dataset, oversampled_indices)
balanced_dataset = torch.utils.data.ConcatDataset([dataset, oversampled_dataset])

# Define DataLoader for the balanced dataset
train_loader = DataLoader(balanced_dataset, batch_size=32, shuffle=True)

# Load pre-trained Inception v3 model
model = inception_v3(pretrained=True)

# Freeze layers
for param in model.parameters():
    param.requires_grad = False

# Modify the fully connected layer to match the number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Early stopping
best_val_loss = float('inf')
patience = 3
counter = 0

# Train the model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs, _ = model(inputs)  # Extract primary logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {100.*correct/total:.2f}%")

    # Validate the model
    # (Note: Without a separate validation dataset, you can skip this part or use cross-validation)
    
# Save the trained model
torch.save(model.state_dict(), 'inception_v3_model.pth')


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import inception_v3
import pandas as pd
from PIL import Image
import os
import torch.nn as nn  # Import nn module directly

class CustomTestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_filenames = [filename for filename in os.listdir(root) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.root, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name  # Return image_name as the image ID

# Define transform for test images
transform_test = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionV3 expects 299x299 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = CustomTestDataset(root="test/images/", transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the trained InceptionV3 model
model = inception_v3(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Assuming 2 output classes (0 and 1)
model.load_state_dict(torch.load('inception_v3_model.pth'))
model.eval()

# Create a DataFrame to store predictions
predictions = {'image_id': [], 'label': []}

# Predict labels for test images
with torch.no_grad():
    for inputs, image_names in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for pred, image_name in zip(predicted, image_names):
            predictions['image_id'].append(image_name.split('.')[0])  # Extract image ID from the file name
            predictions['label'].append(pred.item())

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save predictions to CSV file
df_predictions.to_csv('submission_inception_v3.csv', index=False)


# Your score for this submission is : 0.5826018090692893.

In [2]:
# Mode Based Approach
import pandas as pd

# Load the data from the files
file_paths = ["submission.csv", "submission_1 (1).csv", "submission_2.csv",
"submission_inception_v3.csv","submission_resnet50.csv"]
data_frames = [pd.read_csv(file) for file in file_paths]

# Concatenate the data frames
merged_df = pd.concat(data_frames)

# Group by image_id and calculate the mode label for each image_id
mode_labels = merged_df.groupby('image_id')['label'].agg(lambda x: x.mode()[0])

# Create a new DataFrame with image_id and mode label
result_df = pd.DataFrame({'image_id': mode_labels.index, 'label': mode_labels.values})

# Save the result to a new CSV file
result_df.to_csv('result_2.csv', index=False)


# Score -0.71

# Model -6 Dense Net 169

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import densenet169
import numpy as np

# Define transform to preprocess and augment the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
dataset = ImageFolder(root="images/", transform=transform)

# Count occurrences of each class
class_counts = np.bincount(dataset.targets)
num_classes = len(class_counts)

# Determine the minority class
minority_class = np.argmin(class_counts)

# Oversample the minority class to balance the dataset
oversampled_indices = []
for idx, (data, target) in enumerate(dataset):
    if target == minority_class:
        oversampled_indices.append(idx)

oversampled_dataset = torch.utils.data.Subset(dataset, oversampled_indices)
balanced_dataset = torch.utils.data.ConcatDataset([dataset, oversampled_dataset])

# Define DataLoader for the balanced dataset
train_loader = DataLoader(balanced_dataset, batch_size=32, shuffle=True)

# Load pre-trained DenseNet169 model
model = densenet169(pretrained=True)

# Freeze layers
for param in model.parameters():
    param.requires_grad = False

# Modify the fully connected layer to match the number of classes in your dataset
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Early stopping
best_val_loss = float('inf')
patience = 3
counter = 0

# Train the model
num_epochs = 12
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {100.*correct/total:.2f}%")

    # Validate the model
    # (Note: Without a separate validation dataset, you can skip this part or use cross-validation)
    
# Save the trained model
torch.save(model.state_dict(), 'densenet169_model.pth')


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import densenet169
import pandas as pd
from PIL import Image
import os
import torch.nn as nn  # Import nn module directly

class CustomTestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_filenames = [filename for filename in os.listdir(root) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.root, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name  # Return image_name as the image ID

# Define transform for test images
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match DenseNet169 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = CustomTestDataset(root="test/images/", transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the trained DenseNet169 model
model = densenet169(pretrained=False)
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 2)  # Assuming 2 output classes (0 and 1)
model.load_state_dict(torch.load('densenet169_model.pth'))
model.eval()

# Create a DataFrame to store predictions
predictions = {'image_id': [], 'label': []}

# Predict labels for test images
with torch.no_grad():
    for inputs, image_names in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for pred, image_name in zip(predicted, image_names):
            predictions['image_id'].append(image_name.split('.')[0])  # Extract image ID from the file name
            predictions['label'].append(pred.item())

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save predictions to CSV file
df_predictions.to_csv('submission_densenet169.csv', index=False)


In [8]:
#Mode Based Approach
# Mode Based Approach
import pandas as pd

# Load the data from the files
file_paths = ["submission_2.csv","submission_resnet50.csv","submission_densenet169.csv","submission_vgg19_hope.csv"]
data_frames = [pd.read_csv(file) for file in file_paths]

# Concatenate the data frames
merged_df = pd.concat(data_frames)

# Group by image_id and calculate the mode label for each image_id
mode_labels = merged_df.groupby('image_id')['label'].agg(lambda x: x.mode()[0])

# Create a new DataFrame with image_id and mode label
result_df = pd.DataFrame({'image_id': mode_labels.index, 'label': mode_labels.values})

# Save the result to a new CSV file
result_df.to_csv('result_3.csv', index=False)


# No improvement

# Model 7 VGG-19 

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Define transform to preprocess and augment the images
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
dataset = ImageFolder(root="images/", transform=transform_train)

# Split dataset into train and validation sets
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# Load pre-trained VGG-19 model
model = models.vgg19(pretrained=True)

# Freeze layers
for param in model.parameters():
    param.requires_grad = False

# Replace the classifier
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # 2 output classes (0 and 1)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)  # Only optimize classifier parameters

# Train the model
num_epochs = 10
best_f1 = 0.0
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate F1 score
    val_f1 = f1_score(true_labels, predictions)

    # Save the model with the best F1 score
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'vgg19_model_fine_tuned_with_augmentation.pth')

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation F1 Score: {val_f1:.4f}')

print('Training finished.')



In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vgg19
import pandas as pd
from PIL import Image
import os
import torch.nn as nn  # Import nn module directly

class CustomTestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_filenames = [filename for filename in os.listdir(root) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.root, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name  # Return image_name as the image ID

# Define transform for test images
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match VGG19 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = CustomTestDataset(root="test/images/", transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the trained VGG19 model
model = vgg19(pretrained=False)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # Assuming 2 output classes (0 and 1)
model.load_state_dict(torch.load('vgg19_model_fine_tuned_with_augmentation.pth'))
model.eval()

# Create a DataFrame to store predictions
predictions = {'image_id': [], 'label': []}

# Predict labels for test images
with torch.no_grad():
    for inputs, image_names in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for pred, image_name in zip(predicted, image_names):
            predictions['image_id'].append(image_name.split('.')[0])  # Extract image ID from the file name
            predictions['label'].append(pred.item())

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save predictions to CSV file
df_predictions.to_csv('submission_vgg19_hope.csv', index=False)


# Model VGG With better augmentation

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

# Define transform to preprocess and augment the images
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset
dataset = ImageFolder(root="images/", transform=transform_train)

# Split dataset into train and validation sets
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# Calculate class weights
class_counts = np.bincount([dataset.targets[i] for i in train_indices])
total_samples = len(train_dataset)
class_weights = torch.tensor([total_samples / (len(class_counts) * count) for count in class_counts], dtype=torch.float)

# Load pre-trained VGG-19 model
model = models.vgg19(pretrained=True)

# Freeze layers
for param in model.parameters():
    param.requires_grad = False

# Replace the classifier
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # 2 output classes (0 and 1)

# Define loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define optimizer
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.0001)  # Only optimize classifier parameters

# Train the model
num_epochs = 20
best_f1 = 0.0
for epoch in range(num_epochs):
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    
    # Training phase
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation phase
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate F1 score
    val_f1 = f1_score(true_labels, predictions)

    # Save the model with the best F1 score
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'vgg19_model_fine_tuned_with_augmentation.pth')

    print(f'Validation F1 Score: {val_f1:.4f}')

print('Training finished.')


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vgg19
import pandas as pd
from PIL import Image
import os
import torch.nn as nn  # Import nn module directly

class CustomTestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_filenames = [filename for filename in os.listdir(root) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.root, image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_name  # Return image_name as the image ID

# Define transform for test images
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match VGG19 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = CustomTestDataset(root="test/images/", transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the trained VGG19 model
model = vgg19(pretrained=False)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # Assuming 2 output classes (0 and 1)
model.load_state_dict(torch.load('vgg19_model_fine_tuned_with_augmentation.pth'))
model.eval()

# Create a DataFrame to store predictions
predictions = {'image_id': [], 'label': []}

# Predict labels for test images
with torch.no_grad():
    for inputs, image_names in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for pred, image_name in zip(predicted, image_names):
            predictions['image_id'].append(image_name.split('.')[0])  # Extract image ID from the file name
            predictions['label'].append(pred.item())

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save predictions to CSV file
df_predictions.to_csv('submission_vgg19_hope_1.csv', index=False)


# Last Mode Approach 

In [7]:
import pandas as pd

# Load the data from the files
file_paths = ["submission_2.csv",
              "submission_vgg19_hope_1.csv","submission_vgg19_hope.csv","submission.csv"]
data_frames = [pd.read_csv(file) for file in file_paths]

# Concatenate the data frames
merged_df = pd.concat(data_frames)

# Group by image_id and take the minimum label for each image_id
min_labels = merged_df.groupby('image_id')['label'].max()

# Create a new DataFrame with image_id and minimum label
result_df = pd.DataFrame({'image_id': min_labels.index, 'label': min_labels.values})

# Save the result to a new CSV file
result_df.to_csv('result_minimum.csv', index=False)
